In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import yfinance as yf

In [2]:
comp = ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'NVDA']
data = yf.download(comp, start='2020-01-01', end='2021-01-01', interval='1d')

[*********************100%%**********************]  5 of 5 completed


In [3]:
data = data.dropna()
data.shape

(253, 30)

In [4]:
close = data.Close.values
close.shape

(253, 5)

In [5]:
cov = np.cov(close,rowvar=False)
cov.shape

(5, 5)

In [6]:
def obj(w, V=cov):
    '''
    Construct the objective funcion
    '''
    w = np.array(w)
    sigma = np.sqrt(w.T @ V @ w)
    return np.sum((w*V@w - sigma/len(w))**2)

# Define the inequality constraint w_i >= 0 for all weights w_i
ineq_cons = {'type': 'ineq', 'fun': lambda w: w+1e-9}

# Define the equality constraint sum(w_i) = 1
eq_cons = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

# Initial guess should sum to 1
x0 = np.ones(len(cov)) / len(cov)

# Run the minimization using SLSQP method
result = minimize(obj, x0=x0, method='SLSQP', constraints=[ineq_cons, eq_cons], options={'ftol': 1e-9, 'disp': True})

print(result.x)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 58267.29698772918
            Iterations: 13
            Function evaluations: 87
            Gradient evaluations: 12
[0.16899206 0.46696287 0.17046558 0.12777259 0.0658069 ]
